In [268]:
import sys, pdb, warnings, scipy, matplotlib, sklearn, itertools

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cPickle as pkl
import seaborn as sns

from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFECV

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
sys.setrecursionlimit(15000)
pd.set_option('display.max_columns', 300)
%matplotlib inline

print('scipy: {}'.format(scipy.__version__))
print('numpy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(matplotlib.__version__)) 
print('sklearn: {}'.format(sklearn.__version__))

#our modules see: CS_Project/cspython directory
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped
import cspython.analysis as a
from cspython.merging_processing import combine_dfs

scipy: 0.19.1
numpy: 1.13.3
pandas: 0.22.0
matplotlib: 2.1.0
sklearn: 0.19.1


In [141]:
"""
with open('../cspython/esl_teams.pkl', 'rb') as f: 
     d = pkl.load(f)
        
big_data = process_scrapped(d)
data = combine_dfs(*big_data)
"""

with open('big_data.pkl', 'rb') as f:
    big_data = pkl.load(f)
    
data = combine_dfs(*big_data)
data.sort_values('date', inplace=True)
data.index = range(len(data))
overview = big_data[0]

In [142]:
def create_opponent_team_col(data):
    data.loc[:,'player_team_opponent'] = np.nan
    data.loc[(data['team_A_name'] != data['player_team_name']),'player_team_opponent'] = data.loc[:,'team_A_name']
    data.loc[(data['team_B_name'] != data['player_team_name']),'player_team_opponent'] = data.loc[:,'team_B_name']
    return data

def create_fwa_dr_columns(data, col_list):  # first k , awp, who, divided by rounds 
    columns = pd.Series(data.columns)
    for a in col_list:
        col = columns[columns.str.contains(a)]
        data[a+'_sum_dr'] = data[col].convert_objects(convert_numeric = True).sum(axis = 1) / (data['team_A_score'] + data['team_B_score'])*100
    data.loc[:, data.columns != 'date'] = data.loc[:, data.columns != 'date'].apply(pd.to_numeric, errors='ignore')
    return data

def create_player_columns(data):
    names = data.nicknames.unique()
    for a in names:
        data.loc[:,a] = 0
        data.loc[data.loc[:,'nicknames'] == a, a] = 5   # its 5 so that when you group by team it becomes 1 
    return data
    
def apply_nummeric_and_group_as_match(data):
    r = data.loc[:, data.columns != 'date'].apply(pd.to_numeric, errors='ignore')
    r['date'] = data.date
    data = r
    data = data.fillna(0)
    data_match = data.groupby(['match_id', 'player_team_name', 'date', 'team_A_name', 'team_B_name', 'series_id', 'map', 'winner_of_match', 'loser_of_match','player_team_opponent']).mean()
    data_match = pd.DataFrame(data_match)
    data_match = data_match.reset_index()
    return data_match

def player_based_column_making(data): # these are the functions that are non historic and arent grouped by match_id
    col_list = ['first_kills', 'who_kill_who', 'awp_kills']
    data = create_opponent_team_col(data)
    data = create_fwa_dr_columns(data, col_list)
    data = create_player_columns(data)
    return data

def create_historic_data(data):
    col_list = ['first_kills', 'who_kill_who','awp_kills']
    data = create_fwadr_his(data, col_list)
    data = create_matches_count(data)
    data = create_avdamage_his(data)
    data = create_avdamage_map_his(data)
    data = create_faw_map_his(data, col_list)
    return data

def create_matches_count(data): # how many matches a team has played
    teams = list(data.player_team_name.unique())
    new_group = pd.DataFrame()
    for a in teams:   
        data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by='date',ascending=True)
        grouping = data_team.groupby(['player_team_name','date','match_id'])['ADR'].count()
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        grouping.loc[:,'ADR'] = grouping.loc[:,'ADR'].expanding(min_periods=1, freq=None, center=False, axis=0).sum()
        grouping = grouping.rename(index=str, columns={'ADR': 'matches_played_team'})
        new_group = pd.concat([new_group, grouping])
    data = pd.merge(data, new_group, on = ['player_team_name','match_id','date'])

    return data

def create_avdamage_his(data):  # column with historic average damage of individual !!!
    teams = list(data.player_team_name.unique())
    new_group = pd.DataFrame()
    for a in teams:   
        data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by='date',ascending=True)
        grouping = data_team.groupby(['player_team_name','date','match_id'])['ADR'].max()
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        grouping.loc[:,'ADR'] = grouping.loc[:,'ADR'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
        grouping = grouping.rename(index=str, columns={'ADR': 'ADR_hist'})
        new_group = pd.concat([new_group, grouping])
    data = pd.merge(data, new_group, on = ['player_team_name','match_id','date'])

    return data

def create_avdamage_map_his(data):# historic average damage of individual for each map !!!
    teams = list(data.player_team_name.unique())
    maps = list(data.map.unique())
    new_group = pd.DataFrame()
    for a in maps:
        for b in teams:   
            data_team = data.loc[(data.loc[:,'player_team_name'] == b) & (data.loc[:,'map'] == a), ].sort_values(by='date',ascending=True)
            grouping = data_team.groupby(['player_team_name','match_id'])['ADR'].max()
            grouping = pd.DataFrame(grouping)
            grouping = grouping.reset_index()
            grouping.loc[:,'ADR'] = grouping.loc[:,'ADR'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
            grouping = grouping.rename(index=str, columns={'ADR': 'ADR_hist_on_map'})
            new_group = pd.concat([new_group, grouping])
    
    data = pd.merge(data, new_group, on = ['match_id','player_team_name'])

    return data

def create_fwadr_his(data, col_list):
    teams = list(data.player_team_name.unique())
    for b in col_list:
        new_group = pd.DataFrame()
        for a in teams:   
            data_team = data.loc[data.loc[:,'player_team_name'] == a, ].sort_values(by='date',ascending=True)
            grouping = data_team.groupby(['player_team_name','date','match_id'])[b+'_sum_dr'].max()
            grouping = pd.DataFrame(grouping)
            grouping = grouping.reset_index()
            grouping.loc[:,b +'_sum_dr'] = grouping.loc[:,b +'_sum_dr'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
            grouping = grouping.rename(index=str, columns={b +'_sum_dr': b + '_sum_dr_hist'})
            new_group = pd.concat([new_group, grouping])
        data = pd.merge(data, new_group, on = ['player_team_name','match_id','date'])
    return data

def create_faw_map_his(data, col_list):
    teams = list(data.player_team_name.unique())
    maps = list(data.map.unique())
    for b in col_list:
        new_group = pd.DataFrame()
        for i in maps:
            for a in teams:   
                data_team = data.loc[(data.loc[:,'player_team_name'] == a) & (data.loc[:,'map'] == i), ].sort_values(by='date',ascending=True)
                grouping = data_team.groupby(['player_team_name','match_id'])[b+'_sum_dr'].max()
                grouping = pd.DataFrame(grouping)
                grouping = grouping.reset_index()
                grouping.loc[:,b +'_sum_dr'] = grouping.loc[:,b +'_sum_dr'].expanding(min_periods=1, freq=None, center=False, axis=0).mean()
                grouping = grouping.rename(index=str, columns={b +'_sum_dr': b + '_sum_dr_hist_on_map'})
                new_group = pd.concat([new_group, grouping])
                
        data = pd.merge(data, new_group, on = ['match_id','player_team_name'])
    return data   

def match_dataset_creation(data):  #creates player based columns, then groups to allow for historic match based columns
    data = player_based_column_making(data)
    data = apply_nummeric_and_group_as_match(data)
    return data

In [291]:
"""
data = match_dataset_creation(data)
data.to_pickle("match_dataset.pkl")
"""

data = pd.read_pickle("match_dataset.pkl")
print data.shape


(1208, 440)


In [231]:
def count_map_win_loss_total(df): #this function assumes the df contains only rows from one team
    team_name = df.player_team_name.iloc[0]
    maps = df.map.unique()
    
    for map_name in maps:
        df[map_name + "_win_his"] = np.nan
        df[map_name + "_loss_his"] = np.nan
        df[map_name + "_total_played"] = np.nan
        
    grouped = df.groupby('map')
    for map_name, map_df in grouped:
        played = pd.Series(range(1,len(map_df)+1), index=map_df.index)
        won = (map_df.winner_of_match == team_name).expanding(1).sum()
        lost = played - won
        
        df.loc[map_df.index, map_name + '_total_played'] = played
        df.loc[map_df.index, map_name + '_win_his'] = won
        df.loc[map_df.index, map_name + '_loss_his'] = lost
        
        df.loc[:, map_name + '_total_played'].fillna(method='ffill', inplace=True)
        df.loc[:, map_name + '_win_his'].fillna(method='ffill', inplace=True)
        df.loc[:, map_name + '_loss_his'].fillna(method='ffill', inplace=True)
        
        df.loc[:, map_name + '_total_played'].fillna(0, inplace=True)
        df.loc[:, map_name + '_win_his'].fillna(0, inplace=True)
        df.loc[:, map_name + '_loss_his'].fillna(0, inplace=True)
        
    return df


def create_map_win_loss_and_per_his_columns(data):  # team total win and loses on map with total times played on map !4!
    data.sort_values('date', inplace=True)
    data.index = range(len(data))
    original_col_order = data.columns.tolist()
    data = data.groupby('player_team_name').apply(count_map_win_loss_total)
    new_cols = data.columns[~data.columns.isin(original_col_order)].tolist()
    data = data.loc[:, original_col_order + new_cols]
    for map_name in data.map.unique():
        data.loc[:, map_name + '_win_perc_map'] = data.loc[:, map_name + '_win_his']/data.loc[:, map_name + '_total_played']
    return data


In [ ]:
#  FUNCTIONS BELOW HERE NEED TO BE WORKED ON

In [295]:
#Rounds won and lost in current match
def create_rounds_won_rounds_loss_columns(data):
    team_A_rounds = data.loc[data.player_team_name == data.team_A_name, 'team_A_score']
    team_B_rounds = data.loc[data.player_team_name == data.team_B_name, 'team_B_score']
    rounds_won = pd.concat([team_A_rounds, team_B_rounds])
    data.loc[:, 'rounds_won'] = rounds_won
    
    team_A_rounds = data.loc[data.player_team_name == data.team_B_name, 'team_A_score']
    team_B_rounds = data.loc[data.player_team_name == data.team_A_name, 'team_B_score']
    rounds_lost = pd.concat([team_A_rounds, team_B_rounds])
    data.loc[:, 'rounds_lost'] = rounds_lost
    
    return data

#running total of rounds won/lost vs opponent

def count_rounds_won_vs_opponent(df): #parsing a df of 1 team
    grouped = df.groupby('player_team_opponent')
    for opponent, opponent_df in grouped:
        won_his = opponent_df.loc[:, 'rounds_won'].expanding(1).sum()
        loss_his = opponent_df.loc[:,'rounds_lost'].expanding(1).sum()
        
        df.loc[opponent_df.index, 'rounds_won_vs_'+opponent] = won_his
        df.loc[opponent_df.index, 'rounds_loss_vs_'+opponent] = loss_his
        
        df.loc[:, 'rounds_won_vs_'+opponent].fillna(method='ffill', inplace=True)
        df.loc[:, 'rounds_loss_vs_'+opponent].fillna(method='ffill', inplace=True)
        
        df.loc[:, 'rounds_won_vs_'+opponent].fillna(0, inplace=True)
        df.loc[:, 'rounds_loss_vs_'+opponent].fillna(0, inplace=True)
    return df


def create_rounds_won_and_lost_vs_team_his(data): #applied to entire dataset
    data.sort_values('date', inplace = True)
    data = data.groupby('player_team_name').apply(count_rounds_won_vs_opponent)
    return data


def count_rounds_won_vs_opponent_on_map(df): # parsing a df of 1 team
    opponent = df.player_team_opponent.values[0]
    map_name = df.map.values[0]

    won_his = df.loc[:, 'rounds_won'].expanding(1).sum()
    loss_his = df.loc[:,'rounds_lost'].expanding(1).sum()
        
    df.loc[:, 'rounds_won_vs_'+opponent+'_on_'+map_name] = won_his
    df.loc[:, 'rounds_loss_vs_'+opponent+'_on_'+map_name] = loss_his

    return df


def create_rounds_won_and_lost_vs_team_by_map_his(data):
    data.sort_values('date', inplace = True)
    data = data.groupby(['player_team_name', 'player_team_opponent', 'map']).apply(count_rounds_won_vs_opponent_on_map)
    
    for team, opponent, map_name in itertools.product(data.player_team_name.unique(), data.player_team_opponent.unique(), data.map.unique()):
        if 'rounds_won_vs_'+opponent+'_on_'+map_name not in data.columns:
            data.loc['rounds_won_vs_'+opponent+'_on_'+map_name] = 0
            data.loc['rounds_loss_vs_'+opponent+'_on_'+map_name] = 0
            continue
        col = data.loc[data.player_team_name == team, 'rounds_won_vs_'+opponent+'_on_'+map_name].fillna(method='ffill')
        data.loc[data.player_team_name == team, 'rounds_won_vs_'+opponent+'_on_'+map_name] = col
        col = data.loc[data.player_team_name == team, 'rounds_loss_vs_'+opponent+'_on_'+map_name].fillna(method='ffill')
        data.loc[data.player_team_name == team, 'rounds_loss_vs_'+opponent+'_on_'+map_name] = col
        col = data.loc[data.player_team_name == team, 'rounds_won_vs_'+opponent+'_on_'+map_name].fillna(0)
        data.loc[data.player_team_name == team, 'rounds_won_vs_'+opponent+'_on_'+map_name] = col
        col = data.loc[data.player_team_name == team, 'rounds_loss_vs_'+opponent+'_on_'+map_name].fillna(0)
        data.loc[data.player_team_name == team, 'rounds_loss_vs_'+opponent+'_on_'+map_name] = col
    return data
    
    
def create_round_his_cols(data):
    original_col_order = data.columns.tolist()
    data = create_rounds_won_rounds_loss_columns(data)
    data = create_rounds_won_and_lost_vs_team_his(data)
    data = create_rounds_won_and_lost_vs_team_by_map_his(data)
    new_cols = data.columns[~data.columns.isin(original_col_order)].tolist()
    data = data.loc[:, original_col_order + new_cols]
    return data

In [2]:
import pandas as pd
df = pd.read_pickle('aggregated_dataset.pkl')

In [296]:
tester = create_map_win_loss_and_per_his_columns(data)
tester = create_round_his_cols(tester)

In [3]:
df.to_excel('aggregated_dataset.xlsx')

In [4]:
df.shape

(1216, 686)

In [299]:
tester.loc[(tester.player_team_name == 'NRG')&(tester.player_team_opponent=='SK'), 'map'].unique()

array(['Mirage', 'Cache', 'Cobblestone'], dtype=object)

In [298]:
tester.loc[tester.player_team_name == 'NRG','rounds_loss_vs_SK_on_Overpass']

20      0.0
19      0.0
21      0.0
69      0.0
70      0.0
93      0.0
96      0.0
100     0.0
103     0.0
112     0.0
114     0.0
123     0.0
126     0.0
127     0.0
131     0.0
133     0.0
202     0.0
204     0.0
207     0.0
206     0.0
236     0.0
260     0.0
265     0.0
307     0.0
311     0.0
318     0.0
319     0.0
341     0.0
343     0.0
345     0.0
       ... 
826     0.0
828     0.0
830     0.0
856     0.0
859     0.0
904     0.0
906     0.0
909     0.0
912     0.0
910     0.0
913     0.0
977     0.0
980     0.0
989     0.0
992     0.0
1020    0.0
1026    0.0
1042    0.0
1047    0.0
1054    0.0
1061    0.0
1078    0.0
1113    0.0
1115    0.0
1132    0.0
1141    0.0
1144    0.0
1184    0.0
1185    0.0
1186    0.0
Name: rounds_loss_vs_SK_on_Overpass, Length: 99, dtype: float64

In [237]:
tester.loc[(tester.player_team_name == 'NRG')&(tester.player_team_opponent=='CLG')].to_excel('map_team_breakdown_sanity_check.xlsx')

In [224]:
def create_rounds_won_vs_team_his(data): # team rounds won vs another team    !6!
    grouping = data.groupby(['match_id','map','team_A_name','team_B_name', 'team_A_score', 'team_B_score'])['round_num'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    grouping = grouping.groupby(['team_A_name', 'team_B_name']).sum()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    forward = grouping.team_A_name+grouping.team_B_name
    reverse = grouping.team_B_name+grouping.team_A_name
    for idx, val in enumerate(forward):
        for idx2, val2 in enumerate(reverse):
            if val == val2 and idx < idx2:
                grouping.loc[idx,'team_A_score_Count'] += grouping.loc[idx2,'team_B_score_Count']
                grouping.loc[idx,'team_B_score_Count'] += grouping.loc[idx2,'team_A_score_Count']
                grouping.loc[idx2,'team_B_score_Count'] = grouping.loc[idx,'team_A_score_Count']
                grouping.loc[idx2,'team_A_score_Count'] = grouping.loc[idx,'team_B_score_Count']
            elif val == val2 and idx > idx2:
                grouping.loc[idx2,'team_B_score_Count'] = grouping.loc[idx,'team_A_score_Count']
                grouping.loc[idx2,'team_A_score_Count'] = grouping.loc[idx,'team_B_score_Count']
    
    grouping = grouping.drop('round_num_Count_Count', axis = 1)
    col1 = list(grouping.team_A_name.unique())
    col2  = list(grouping.team_B_name.unique())
    col = col1 + col2
    col = list(set(col))
    data = pd.merge(data,grouping, on=['team_A_name', 'team_B_name']) 
    for a in col:
        data['rd_total_his_'+ a] = 0
        data.loc[(data.player_team_name != a) & (data.team_A_name == a) , 'rd_total_his_'+ a]=data.team_B_score_Count
        data.loc[(data.player_team_name != a) & (data.team_B_name == a) , 'rd_total_his_'+ a]=data.team_A_score_Count
        bgrouping = data.groupby(['player_team_name'])['rd_total_his_'+ a].max()
        bgrouping = pd.DataFrame(bgrouping)
        bgrouping = bgrouping.reset_index()
        data = data.drop('rd_total_his_'+ a, axis = 1)
        data = pd.merge(data, bgrouping, on = 'player_team_name')
       
    return data    

In [225]:
def create_total_team_rd_map_his(data):
    grouping = data.groupby(['map','team_A_name', 'team_A_score'])['round_num'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    grouping = grouping.groupby(['map','team_A_name'])[ 'team_A_score'].sum()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    fgrouping = data.groupby(['player_team_name','map','team_B_name', 'team_B_score'])['round_num'].count()
    fgrouping = pd.DataFrame(fgrouping)
    fgrouping = fgrouping.add_suffix('_Count').reset_index()
    fgrouping = fgrouping.groupby(['map','team_B_name'])[ 'team_B_score'].sum()
    fgrouping = pd.DataFrame(fgrouping)
    fgrouping = fgrouping.add_suffix('_Count').reset_index()
    fgrouping = fgrouping.rename(index=str, columns={"team_B_name": "team_A_name", 'team_B_score_Count': 'team_A_score_Count'})
    merged = pd.concat([grouping, fgrouping], axis = 0)
    merged.groupby(['map', 'team_A_name'])['team_A_score_Count'].sum()
    merged = pd.DataFrame(merged)
    merged = merged.reset_index(drop = True)
    merged = merged.rename(index=str, columns={"team_A_name": "player_team_name", "team_A_score_Count": 'total_team_rd_map'})
    merged = merged.groupby(['player_team_name', 'map']).sum()
    merged = pd.DataFrame(merged)
    merged = merged.reset_index()
    for a in list(merged.map.unique()):
        merged.loc[:,'total_team_rd_'+ a] = 0
        merged.loc[(merged.loc[:, 'map'] == a), 'total_team_rd_'+ a] = merged.loc[:,'total_team_rd_map']
        ok_map = merged.groupby(['player_team_name'])['total_team_rd_'+ a].max()
        ok_map = pd.DataFrame(ok_map)
        ok_map = ok_map.reset_index()
        data = pd.merge(data, ok_map, on = 'player_team_name')
    return data
    

In [ ]:
#FUNCTIONS ABOVE HERE NEED TO BE WORKED ON

In [230]:
#data.to_pickle("data_converged_changed.pkl")


In [229]:
col_list = ['first_kills', 'who_kill_who','awp_kills']
data = create_fwa_dr_columns(data, col_list)
data = create_fwadr_his(data,col_list)
data = create_avdamage_his(data)
data = create_map_win_loss_his(data)
data = create_map_win_his_per(data)
data = create_rounds_won_vs_team_his(data)
data = create_total_team_rd_map_his(data)
data = create_avdamage_map_his(data)
data = create_faw_map_his(data, col_list)
data = create_matches_count(data)
data = create_opponent_team_col(data)